<a href="https://colab.research.google.com/github/tono000/CS114.K11/blob/master/Feature_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import cv2
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imutils import paths
import numpy as np
import pickle
import random



In [0]:
from google.colab import drive
drive.mount('/mydrive',True)

Mounted at /mydrive


In [0]:
path = '/mydrive/My Drive/hog_R_CNN_dataset'
folder = path
print(folder)

/mydrive/My Drive/hog_R_CNN_dataset


In [0]:
model = VGG16(weights = 'imagenet', include_top = False)
model.summary()

In [0]:
#Tạo dataframe labels
dem=0
data=[0,0]
X = pd.DataFrame(columns=range(224*224))
Y = pd.DataFrame(columns=range(1))
from google.colab.patches import cv2_imshow

for i in range(0,2):
  temp_arr=[]
  temp_label=[]
  branch_folder = folder + "/" + os.listdir(folder)[i]
  for filenames in os.listdir(branch_folder):
          print(dem)
          img=cv2.imread(branch_folder+'/'+filenames)
          print(branch_folder+'/'+filenames)
          imgCrop=img[0:((img.shape[0]-1)//2),]     
          imgCrop=cv2.resize(imgCrop, (224,224))
          arr=imgCrop.flatten()   
          temp_arr.append(arr)
          temp_label.append(1-i)
          dem=dem+1
          
  t=pd.DataFrame(data = temp_arr)
  X=X.append(t,ignore_index=True)
  Y=Y.append(temp_label,ignore_index=True)    


In [0]:
le = LabelEncoder()
labels = le.fit_transform(Y)
labels=pd.DataFrame(data = labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#Load ảnh
list_image = []
from google.colab.patches import cv2_imshow
for i in range(0,2):
  temp_arr=[]
  branch_folder = folder + "/" + os.listdir(folder)[i]
  for imagePath in os.listdir(branch_folder):

      #resize ảnh về 224x224 pixels
      
        image = load_img(branch_folder+"/"+imagePath,color_mode='rgb',target_size=(448,224))
        
        print(imagePath)
        print(image)
        
        image = img_to_array(image)
        image=image[0:224,]

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = preprocess_input(image)
        cv2_imshow(image)
        image = np.expand_dims(image, axis = 0)
        list_image.append(image)      
    
      
list_image = np.vstack(list_image)

In [0]:
#Lấy feature từ lớp feature extract của VGG
features = model.predict(list_image)
features = features.reshape((features.shape[0], 512*7*7))

In [0]:
#plit tỉ lệ 80/20
from keras.utils.np_utils import to_categorical
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [0]:
#Train
params = {'C' : [0.1, 1.0, 10.0, 100.0]}
LogReg = GridSearchCV(LogisticRegression(), params)
LogReg.fit(x_train, y_train)

In [0]:
#Evaluate trên test
from sklearn.metrics import classification_report
y_pred=LogReg.predict(x_test)

print(y_pred)

print(y_test.values.ravel())
print(y_test.index.values.ravel())
print(classification_report(y_test, y_pred))

[1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 0]
[1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0]
[ 69 140  27  19  42 117 126 108  84  18  12  55 128  78  73  36 112 133
 100 101  94 136  11  66  31  45  51  76 111]
              precision    recall  f1-score   support

           0       0.80      0.62      0.70        13
           1       0.74      0.88      0.80        16

    accuracy                           0.76        29
   macro avg       0.77      0.75      0.75        29
weighted avg       0.77      0.76      0.75        29



In [0]:
#Thử evalate lại trên train
from sklearn.metrics import classification_report
y_pred=LogReg.predict(x_train)
print(y_train.index.values.ravel())
print(classification_report(y_train, y_pred))

[  9   4  26 120 125  16  10 118  60  67  98 110  97 114   0  65  28  40
  85  24  25  23  80  68  44  15  96  39  56  22  64  81 134  47  30  89
 132  33  82  93  62  95 119  53   5  86 143  49  35  77  34 104   7  43
  70 113  83 124 131   8  13 109 115   3  17  38  72 137   6 138   2 123
  54  50 105  46 141  61 142  79 122  91  41  58  90  48  88 127  21  57
  75  32 139  59  63 129  37  29 107 135   1  52 130 103  99 116  87  74
 121 144  20  71 106  14  92 102]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        51
           1       1.00      1.00      1.00        65

    accuracy                           1.00       116
   macro avg       1.00      1.00      1.00       116
weighted avg       1.00      1.00      1.00       116

